# partie 3 : modélisation thématique

Julien Velcin, Université Lyon 2 - Master Humanités Numériques

In [1]:
%matplotlib inline

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim import corpora, models
#from gensim.models import  Word2Vec
import gensim
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from matplotlib import cm 
import os

# en utilisant gensim, on a besoin de préciser l'expression qui permet de "tokenizer" les textes
#tokenizer = RegexpTokenizer(r'\w+')

## Preprocessing

On va illustrer l'utilisation du modèle LDA

In [2]:
from gensim.utils import simple_preprocess
import pandas

# on charge les données dans un tableau
#df = pandas.read_csv("datasets/huma1.csv", sep="\t")
#df = pandas.read_csv("datasets/dataconf.csv", sep="\t")

# lecture des données pour un fichier texte simple
with open(os.path.join("datasets", "Frank Herbert - Dune.txt")) as f:
    lines = [line.strip() for line in f.readlines()]
doc_set = lines

# colonne qui contient les textes à analyser
#var_texte = 'text'
#var_texte = 'title'
#doc_set = df[var_texte].tolist()


# fonction qui génère les listes de mots (token) à partir des textes
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# on construit le corpus
data_words = list(sent_to_words(doc_set))

In [3]:
# nombre total de documents
ndocs = len(data_words)
print(ndocs)

8608


## Dictionnaire et corpus

On ne va pas utiliser la librairie *scikit-learn* cette fois, mais passer directement par la librairie *gensim*.

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# on retire les mots-outils
data_words_nostops = remove_stopwords(data_words)

In [5]:
print(data_words_nostops[:1])

[['dune']]


In [6]:
# création du dictionnaire
dico = corpora.Dictionary(data_words_nostops)

# ce qui permet par ex. de filtrer le vocabulaire
dico.filter_extremes(no_below=10)

# Create Corpus
texts = data_words_nostops

# matrice Term Document Frequency
corpus = [dico.doc2bow(text) for text in texts]

Nous pouvons vérifier que l'entrée souhaitée est une liste de documents qui sont, chacun, représentés comme une liste de tuples (identifiant du mot dans le dictionnaire, TF).

In [7]:
print(corpus[:10])

[[(0, 1)], [], [], [], [], [(1, 1)], [(0, 1)], [], [], []]


In [8]:
len(dico)

1809

## Apprentissage du modèle

La plupart du temps, il faut fixer le nombre de thématiques souhaitées.

In [9]:
ntopics = 50
#ntopics = 20

On va utiliser le modèle LDA implémenté dans la librairie *gensim*.

In [10]:
# thanks to: https://miningthedetails.com/blog/python/lda/GensimLDA/
from gensim.models.ldamodel import LdaModel

#generate_lda = True
generate_lda = False

# generate LDA model
import logging

if generate_lda:
    print("generate new LDA model")
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    ldamodel = LdaModel(corpus, num_topics=ntopics, id2word = dico,
                                           passes=100, random_state=100, per_word_topics=True)
    print(ldamodel)


2021-01-15 12:18:15,481 : INFO : using symmetric alpha at 0.02
2021-01-15 12:18:15,492 : INFO : using symmetric eta at 0.02
2021-01-15 12:18:15,498 : INFO : using serial LDA version on this node


generate new LDA model


2021-01-15 12:18:15,557 : INFO : running online (multi-pass) LDA training, 50 topics, 100 passes over the supplied corpus of 8608 documents, updating model once every 2000 documents, evaluating perplexity every 8608 documents, iterating 50x with a convergence threshold of 0.001000
2021-01-15 12:18:15,567 : INFO : PROGRESS: pass 0, at document #2000/8608
2021-01-15 12:18:16,961 : INFO : merging changes from 2000 documents into a model of 8608 documents
2021-01-15 12:18:16,977 : INFO : topic #30 (0.020): 0.022*"halleck" + 0.020*"piter" + 0.020*"said" + 0.017*"water" + 0.014*"room" + 0.014*"life" + 0.012*"could" + 0.011*"kynes" + 0.010*"bene" + 0.010*"gesserit"
2021-01-15 12:18:16,980 : INFO : topic #15 (0.020): 0.052*"paul" + 0.021*"shield" + 0.017*"voice" + 0.011*"could" + 0.011*"said" + 0.010*"mood" + 0.010*"door" + 0.010*"place" + 0.009*"take" + 0.009*"son"
2021-01-15 12:18:16,985 : INFO : topic #49 (0.020): 0.041*"said" + 0.027*"duke" + 0.017*"room" + 0.013*"would" + 0.013*"paul" + 0

2021-01-15 12:18:22,008 : INFO : merging changes from 2000 documents into a model of 8608 documents
2021-01-15 12:18:22,019 : INFO : topic #1 (0.020): 0.053*"atreides" + 0.040*"leto" + 0.036*"duke" + 0.035*"said" + 0.034*"alia" + 0.033*"daughter" + 0.029*"traitor" + 0.026*"thus" + 0.025*"find" + 0.025*"long"
2021-01-15 12:18:22,020 : INFO : topic #26 (0.020): 0.065*"thufir" + 0.053*"whispered" + 0.052*"give" + 0.047*"silence" + 0.036*"hawat" + 0.031*"said" + 0.028*"paul" + 0.028*"tell" + 0.025*"ways" + 0.023*"beginning"
2021-01-15 12:18:22,022 : INFO : topic #3 (0.020): 0.087*"said" + 0.073*"lord" + 0.067*"know" + 0.067*"yes" + 0.032*"ah" + 0.031*"training" + 0.030*"baron" + 0.020*"woman" + 0.020*"believe" + 0.019*"perhaps"
2021-01-15 12:18:22,040 : INFO : topic #13 (0.020): 0.037*"say" + 0.036*"kwisatz" + 0.036*"haderach" + 0.034*"want" + 0.028*"figure" + 0.028*"ring" + 0.026*"per" + 0.023*"cent" + 0.020*"smuggler" + 0.020*"paul"
2021-01-15 12:18:22,078 : INFO : topic #38 (0.020): 0.0

2021-01-15 12:18:26,597 : INFO : topic #22 (0.020): 0.058*"lady" + 0.056*"told" + 0.056*"baron" + 0.054*"arrakis" + 0.050*"jessica" + 0.038*"filled" + 0.037*"never" + 0.032*"memory" + 0.031*"person" + 0.030*"become"
2021-01-15 12:18:26,598 : INFO : topic #24 (0.020): 0.078*"troop" + 0.051*"rabban" + 0.050*"slave" + 0.046*"child" + 0.029*"matter" + 0.025*"knife" + 0.021*"plant" + 0.021*"saw" + 0.020*"metal" + 0.019*"near"
2021-01-15 12:18:26,602 : INFO : topic #23 (0.020): 0.081*"name" + 0.047*"god" + 0.041*"world" + 0.039*"removed" + 0.037*"legend" + 0.034*"faced" + 0.031*"missionaria" + 0.031*"protectiva" + 0.027*"brought" + 0.025*"repeated"
2021-01-15 12:18:26,613 : INFO : topic #45 (0.020): 0.094*"head" + 0.043*"storm" + 0.039*"shook" + 0.034*"bring" + 0.029*"swallowed" + 0.027*"years" + 0.025*"jessica" + 0.025*"stare" + 0.025*"glance" + 0.024*"snapped"
2021-01-15 12:18:26,617 : INFO : topic diff=0.429453, rho=0.370015
2021-01-15 12:18:26,619 : INFO : PROGRESS: pass 2, at document #

...

2021-01-15 12:23:07,019 : INFO : merging changes from 2000 documents into a model of 8608 documents
2021-01-15 12:23:07,024 : INFO : topic #26 (0.020): 0.274*"hawat" + 0.075*"give" + 0.069*"tell" + 0.062*"said" + 0.060*"whispered" + 0.056*"silence" + 0.054*"thufir" + 0.035*"paul" + 0.027*"anger" + 0.024*"beginning"
2021-01-15 12:23:07,029 : INFO : topic #3 (0.020): 0.150*"know" + 0.107*"said" + 0.096*"old" + 0.079*"lord" + 0.074*"woman" + 0.063*"yes" + 0.043*"ah" + 0.033*"perhaps" + 0.032*"young" + 0.028*"sure"
2021-01-15 12:23:07,030 : INFO : topic #4 (0.020): 0.175*"saw" + 0.111*"another" + 0.083*"felt" + 0.041*"paul" + 0.039*"thing" + 0.037*"someone" + 0.035*"jessica" + 0.034*"killed" + 0.033*"watched" + 0.025*"guards"
2021-01-15 12:23:07,042 : INFO : topic #28 (0.020): 0.164*"people" + 0.064*"thopter" + 0.045*"dune" + 0.039*"paul" + 0.036*"hidden" + 0.034*"strength" + 0.032*"five" + 0.028*"remembered" + 0.028*"wait" + 0.027*"dream"
2021-01-15 12:23:07,063 : INFO : topic #8 (0.020):

2021-01-15 12:23:10,616 : INFO : topic #10 (0.020): 0.201*"kynes" + 0.158*"man" + 0.073*"said" + 0.050*"every" + 0.045*"used" + 0.029*"kill" + 0.025*"liet" + 0.022*"everything" + 0.021*"way" + 0.020*"purpose"
2021-01-15 12:23:10,618 : INFO : topic #29 (0.020): 0.300*"see" + 0.140*"feyd" + 0.125*"rautha" + 0.048*"maker" + 0.048*"boy" + 0.032*"part" + 0.029*"went" + 0.025*"found" + 0.022*"upward" + 0.020*"kept"
2021-01-15 12:23:10,621 : INFO : topic #12 (0.020): 0.162*"stilgar" + 0.114*"gesserit" + 0.114*"bene" + 0.059*"said" + 0.057*"count" + 0.046*"nodded" + 0.041*"dead" + 0.033*"fenring" + 0.031*"way" + 0.029*"paul"
2021-01-15 12:23:10,623 : INFO : topic #24 (0.020): 0.072*"rabban" + 0.069*"troop" + 0.067*"child" + 0.066*"knife" + 0.049*"slave" + 0.039*"matter" + 0.031*"plant" + 0.028*"eye" + 0.027*"metal" + 0.027*"near"
2021-01-15 12:23:10,630 : INFO : topic diff=0.019922, rho=0.098868
2021-01-15 12:23:10,633 : INFO : PROGRESS: pass 97, at document #8000/8608
2021-01-15 12:23:11,415 

2021-01-15 12:23:15,655 : INFO : topic #18 (0.020): 0.139*"could" + 0.095*"knew" + 0.063*"new" + 0.058*"training" + 0.056*"sietch" + 0.039*"trained" + 0.038*"gave" + 0.033*"fact" + 0.028*"school" + 0.024*"one"
2021-01-15 12:23:15,657 : INFO : topic #14 (0.020): 0.155*"turned" + 0.064*"away" + 0.047*"beneath" + 0.045*"paul" + 0.040*"back" + 0.034*"robe" + 0.034*"lay" + 0.032*"question" + 0.030*"answer" + 0.027*"chamber"
2021-01-15 12:23:15,661 : INFO : topic diff=0.016567, rho=0.098388
2021-01-15 12:23:15,923 : INFO : -9.339 per-word bound, 647.7 perplexity estimate based on a held-out corpus of 608 documents with 5442 words
2021-01-15 12:23:15,924 : INFO : PROGRESS: pass 98, at document #8608/8608
2021-01-15 12:23:16,091 : INFO : merging changes from 608 documents into a model of 8608 documents
2021-01-15 12:23:16,100 : INFO : topic #41 (0.020): 0.149*"must" + 0.061*"said" + 0.058*"may" + 0.054*"make" + 0.054*"among" + 0.049*"harkonnens" + 0.038*"course" + 0.037*"us" + 0.036*"thought" 

2021-01-15 12:23:19,988 : INFO : topic #21 (0.020): 0.109*"great" + 0.090*"many" + 0.066*"things" + 0.065*"one" + 0.047*"always" + 0.036*"already" + 0.033*"rule" + 0.033*"order" + 0.028*"said" + 0.020*"animal"
2021-01-15 12:23:20,002 : INFO : topic diff=0.037524, rho=0.097915


LdaModel(num_terms=1809, num_topics=50, decay=0.5, chunksize=2000)


On peut sauvegarder le modèle dans un fichier pour le garder en mémoire.

In [19]:
temp_file = "models/model_dataconf_" + str(ntopics)

if generate_lda:
    ldamodel.save(temp_file)
else:
    ldamodel = LdaModel.load(temp_file)    

2021-01-15 12:24:24,168 : INFO : saving LdaState object under models/model_dataconf_50.state, separately None
2021-01-15 12:24:24,183 : INFO : saved models/model_dataconf_50.state
2021-01-15 12:24:24,190 : INFO : saving LdaModel object under models/model_dataconf_50, separately ['expElogbeta', 'sstats']
2021-01-15 12:24:24,202 : INFO : storing np array 'expElogbeta' to models/model_dataconf_50.expElogbeta.npy
2021-01-15 12:24:24,252 : INFO : not storing attribute id2word
2021-01-15 12:24:24,255 : INFO : not storing attribute dispatcher
2021-01-15 12:24:24,290 : INFO : not storing attribute state
2021-01-15 12:24:24,296 : INFO : saved models/model_dataconf_50


In [20]:
pwz = ldamodel.get_topics()

print("On peut récupérer la matrice stockant p(w/z):", pwz.shape)

# on peut aussi utiliser ldamodel.get_topic_terms(topicid, topn=n) pour obtenir
# les top n mots via leur identifiant, accompagnés de la proba p(w/z)

#ldamodel.get_topic_terms(1,topn=len(dico))

On peut récupérer la matrice stockant p(w/z): (50, 1809)


In [21]:
# show_topics permet d'afficher les mots directement
ldamodel.show_topics(num_topics=ntopics,formatted=False)

[(0,
  [('fremen', 0.39084002),
   ('use', 0.059228238),
   ('wish', 0.045204528),
   ('al', 0.038892522),
   ('desert', 0.033557408),
   ('live', 0.028252507),
   ('remember', 0.02324076),
   ('clear', 0.02114224),
   ('one', 0.020718865),
   ('different', 0.018741386)]),
 (1,
  [('duke', 0.2072339),
   ('leto', 0.08744753),
   ('atreides', 0.07339103),
   ('son', 0.07011205),
   ('said', 0.056533955),
   ('alia', 0.049839783),
   ('daughter', 0.033081464),
   ('long', 0.032318622),
   ('one', 0.028043028),
   ('thus', 0.027558316)]),
 (2,
  [('gurney', 0.19005096),
   ('halleck', 0.09838473),
   ('said', 0.06477322),
   ('go', 0.063096724),
   ('called', 0.05653797),
   ('paul', 0.0520262),
   ('made', 0.043887354),
   ('man', 0.029949298),
   ('instant', 0.023104072),
   ('darkness', 0.02247189)]),
 (3,
  [('know', 0.14811775),
   ('said', 0.106028765),
   ('old', 0.09817265),
   ('woman', 0.07485444),
   ('lord', 0.07103217),
   ('yes', 0.061214544),
   ('ah', 0.044197567),
   ('yo

On peut vérifier que les mots ont une probabilité d'appartenir à *plusieurs* thématiques

In [24]:
import numpy as np
tab = ldamodel.get_topics()
myword = dico.token2id['data']
np.where(tab[:,myword]>0.03)

(array([], dtype=int64),)

In [25]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Si on souhaite obtenir p(z|d), il faut réexécuter le modèle sur les données (par ex., le corpus).

Les procédures suivantes fournissent plusieurs "vues" intéressantes sur le modèle. Elles viennent du site *machinelearningplus.com* :

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

Tout d'abord, on souhaite un tableau qui liste la thématique majoritaire pour chaque document, accompagnée par ses mots les plus probables.

In [26]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    i=0
    for i, row in enumerate(ldamodel[corpus]):
        #print(row[0])            
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        #print(row)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, texts=doc_set)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,28.0,0.51,"people, thopter, dune, paul, hidden, strength,...",Dune
1,1,0.0,0.02,"fremen, use, wish, al, desert, live, remember,...",Frank Herbert
2,2,0.0,0.02,"fremen, use, wish, al, desert, live, remember,...",
3,3,0.0,0.02,"fremen, use, wish, al, desert, live, remember,...",Copyright 1965
4,4,0.0,0.02,"fremen, use, wish, al, desert, live, remember,...",
5,5,49.0,0.51,"high, small, presently, family, major, obvious...",Book 1
6,6,28.0,0.51,"people, thopter, dune, paul, hidden, strength,...",DUNE
7,7,0.0,0.02,"fremen, use, wish, al, desert, live, remember,...",
8,8,0.0,0.02,"fremen, use, wish, al, desert, live, remember,...",= = = = = =
9,9,0.0,0.02,"fremen, use, wish, al, desert, live, remember,...",


On peut vouloir obtenir les documents les plus "représentatifs" de chaque thématique (attention, au sens de p(z|d)).

In [27]:
sent_topics_sorteddf = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf = pd.concat([sent_topics_sorteddf, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.7550,"fremen, use, wish, al, desert, live, remember,...","""Lisan al-Gaib!"""
1,1.0,0.7800,"duke, leto, atreides, son, said, alia, daughte...","Jessica spoke bitterly: ""Chips in the path of ..."
2,2.0,0.8367,"gurney, halleck, said, go, called, paul, made,...","And Paul: ""Gurney, man! Gurney, man!"""
3,3.0,0.7550,"know, said, old, woman, lord, yes, ah, young, ...","""When we've rested,"" Jessica said, ""we should ..."
4,4.0,0.6733,"saw, another, felt, paul, thing, someone, jess...","Paul collapsing their tent, recovering it up t..."
5,5.0,0.8040,"mother, reverend, noted, attention, power, oft...","""Reverend Mother!"" Chani said. ""What is wrong?"""
6,6.0,0.7550,"poison, get, hold, message, melange, said, rea...","""I'm perfectly safe here,"" Paul said."
7,7.0,0.7550,"asked, idaho, paul, basin, jessica, green, wil...",Paul cleared his throat.
8,8.0,0.9423,"rock, light, sand, moved, hulud, shai, across,...","Slowly, the filtered sun buried itself beneath..."
9,9.0,0.7550,"guild, yet, imperium, human, slowly, report, m...","""I'm a soldier of the Imperium,"" Paul said, ""t..."


Pour finir, le "volume" estimé de documents (en réalité, de mots) couverts par les différentes thématiques.

In [28]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts

0.0     587
3.0     439
46.0    350
12.0    312
42.0    311
17.0    297
41.0    278
43.0    277
44.0    270
1.0     263
8.0     261
7.0     258
37.0    235
10.0    231
2.0     228
26.0    208
34.0    196
11.0    177
29.0    173
30.0    160
5.0     155
31.0    152
25.0    146
38.0    141
39.0    137
4.0     135
14.0    132
27.0    125
22.0    120
45.0    115
16.0    115
21.0    112
40.0    112
47.0    110
18.0    106
48.0    103
19.0     99
9.0      99
13.0     98
24.0     88
28.0     83
35.0     79
20.0     79
15.0     78
33.0     77
6.0      71
36.0     65
49.0     62
32.0     56
23.0     47
Name: Dominant_Topic, dtype: int64